#### Training on rgb image insted of training on grayscale image.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [2]:
Data_Dir = "C:\\Users\\raj-pc\\Downloads\\Jupyter lab\\Data\\Cats and Dogs Microsoft\\PICS"
categories = ['CATS', 'DOGS']

In [7]:
Img_Size = 64
training_data = []

def create_training_data():
    for category in categories:
        path = os.path.join(Data_Dir, category)  ## Path to access cat or dog dir
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img)) ## Full path to image
                new_img_array = cv2.resize(img_array, (Img_Size, Img_Size))  ## Setting all images to same size
                training_data.append([new_img_array[...,::-1], class_num])
            except Exception as e:
                pass

            
        
create_training_data()
print(len(training_data))

24946


In [8]:
import random
random.shuffle(training_data)

In [9]:
X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)

del training_data

In [11]:
X = np.array(X)
X = X.reshape(-1, Img_Size,Img_Size,3)
y = np.array(y, dtype=np.uint8)
y = y.reshape(y.shape[0], 1)

In [14]:
import pickle

pickle_out = open("X_64_rgb.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y_64_rgb.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

### Model Evaluation

In [15]:
import pickle
X = pickle.load(open("X_64_rgb.pickle", "rb"))
y = pickle.load(open("y_64_rgb.pickle", "rb"))

In [16]:
import numpy as np
X = X/255.0

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)

print(X_train.shape, X_test.shape)
## ip_shape = (X.shape[1],X.shape[2], 1)

(19956, 64, 64, 3) (4990, 64, 64, 3)


In [18]:
del X, y

In [19]:
import tensorflow as tf

In [27]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(16, kernel_size=3, padding = 'same'))
model.add(tf.keras.layers.MaxPool2D((2, 2), strides=2))

model.add(tf.keras.layers.Conv2D(32, kernel_size=3, padding = 'same'))
model.add(tf.keras.layers.MaxPool2D((2, 2), strides=2))

model.add(tf.keras.layers.Conv2D(64, kernel_size=3))
model.add(tf.keras.layers.MaxPool2D((2, 2), strides=2))

model.add(tf.keras.layers.Conv2D(64, kernel_size=3))
model.add(tf.keras.layers.MaxPool2D((2, 2), strides=2))

model.add(tf.keras.layers.Flatten())


model.add(tf.keras.layers.Dense(128, 'relu', kernel_regularizer = "L2"))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(128, 'relu', kernel_regularizer = "L2"))
model.add(tf.keras.layers.Dense(1, 'sigmoid'))

model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])

In [28]:
model.fit(X_train, y_train, epochs = 13, use_multiprocessing = True)

Epoch 1/13
624/624 [==============================] - 14s 22ms/step - loss: 1.4153 - accuracy: 0.5786
Epoch 2/13
624/624 [==============================] - 13s 22ms/step - loss: 0.5931 - accuracy: 0.7050
Epoch 3/13
624/624 [==============================] - 13s 21ms/step - loss: 0.5041 - accuracy: 0.7679
Epoch 4/13
624/624 [==============================] - 13s 21ms/step - loss: 0.4493 - accuracy: 0.8091
Epoch 5/13
624/624 [==============================] - 13s 21ms/step - loss: 0.4062 - accuracy: 0.8344
Epoch 6/13
624/624 [==============================] - 14s 22ms/step - loss: 0.3606 - accuracy: 0.8562
Epoch 7/13
624/624 [==============================] - 13s 22ms/step - loss: 0.3185 - accuracy: 0.8809
Epoch 8/13
624/624 [==============================] - 14s 22ms/step - loss: 0.2909 - accuracy: 0.8959
Epoch 9/13
624/624 [==============================] - 14s 22ms/step - loss: 0.2530 - accuracy: 0.9130
Epoch 10/13
624/624 [==============================] - 14s 22ms/step - loss: 0.214

In [25]:
from sklearn.metrics import f1_score

In [29]:
pred = model.predict(X_test)
pred1 = np.apply_along_axis(lambda x : int(x>0.5), 1, pred)
pred1 = pred1.reshape(pred1.shape[0], 1)
print(f1_score(pred1, y_test))

0.8115766039263307
